## Package

In [1]:
import numpy as np
import math
from scipy import stats

## Problem 1
一大型"新興"經濟體電信股股票歐式陽春型買權，賣出1000股部位。設定如下， \
 請問此部位需要計提多少的"FXR"市場風險資本? \
 又此部位需要計提多少的"GIRR"市場風險資本?

In [2]:
# setting
S = 100   # 股票價格
K = 100   # 執行價格
T = 9/12  # 到期日
# 市場即期利率
r1 = 0.02 ; r3 = 0.0225 ; r6 = 0.025 ; r12 = 0.028
# 隱含波動率
σ6 = 0.25 ; σ12 = 0.3
# 風險權數
RW = 0.60
# 同標的選擇權之相關係數
ρ_delta = 0.999

function

In [3]:
### 內插法
def interpo(x,p1,p2):
    
    k = p1[1]*((p2[0]-x)/(p2[0]-p1[0]))
    m = p2[1]*((x-p1[0])/(p2[0]-p1[0]))
    
    return k+m

### 歐式選擇權價格計算

def Europrice(S,K,r,σ,t):
    
    d1 = (np.log(S/K) + (r + 0.5 * σ**2)*t)/(σ * np.sqrt(t))
    d2 = d1 - σ * np.sqrt(t)
    
    call = S * stats.norm.cdf(d1,0,1) - K * np.exp(-r*t) * stats.norm.cdf(d2,0,1)
    put =  K * np.exp(-r*t) * stats.norm.cdf((-1)*d2,0,1) - S * stats.norm.cdf((-1)*d1,0,1)
    
    return call , put


def ϕ(x,y):
    if (x<0) & (y<0):
        return 0
    else:
        return 1

### EQR Delta

In [4]:
r9 = (1/interpo(9,[6,1/(1+r6)],[12,1/(1+r12)]))-1
σ9 = interpo(9,[6,σ6],[12,σ12])
print("The nine months' risk-free spot rate is",round(r9,4))
print("and implied volatility is",round(σ9,4))

The nine months' risk-free spot rate is 0.0265
and implied volatility is 0.275


In [5]:
call,_ = Europrice(S,K,r9,σ9,T)
dS_call,_ = Europrice(1.01*S,K,r9,σ9,T)
FXR_Δ = (dS_call-call)/0.01
FXR_WS_Δ = RW*FXR_Δ

print("call price =",round(call,4)) ; print()
print('price after change 1% of stock price=',round(dS_call,4)) ; print()
print("Delta =",round(FXR_Δ,4)) ; print()
print("   >>> Weighted Sensitivity =",round(FXR_WS_Δ,4)) ; print()

call price = 10.4018

price after change 1% of stock price= 10.9902

Delta = 58.8401

   >>> Weighted Sensitivity = 35.3041



##### 僅有一個選擇權，故不需計算bucket內or間的彙整

In [6]:
print("   >>> EQR Delta =",round(FXR_WS_Δ,4))

   >>> EQR Delta = 35.3041


### EQR Vega

In [28]:
##changing 6 month's r
d6_σ9 = interpo(9,[6,1.01*σ6],[12,σ12])
dσ_call6,_ = Europrice(S,K,r9,d6_σ9,T)
FXR_ν6 = (dσ_call6-call)/0.01
FXR_WS_ν6 = RW*FXR_ν6*σ9
print("Changing 6 month's r :")
print(' price after change 1% of σ=',round(dσ_call6,4))
print(" Vega =",round(FXR_ν6,4)) 
print("   >>> Weighted Sensitivity =",round(FXR_WS_ν6,4))
print()

##changing 12 month's r
d12_σ9 = interpo(9,[6,1.01*σ6],[12,1.01*σ12])
dσ_call12,_ = Europrice(S,K,r9,d12_σ9,T)
FXR_ν12 = (dσ_call12-call)/0.01
FXR_WS_ν12 = RW*FXR_ν12*σ9
print("Changing 12 month's r :")
print(' price after change 1% of σ=',round(dσ_call12,4)) 
print(" Vega =",round(FXR_ν12,4)) 
print("   >>> Weighted Sensitivity =",round(FXR_WS_ν12,4))

Changing 6 month's r :
 price after change 1% of σ= 10.4441
 Vega = 4.2309
   >>> Weighted Sensitivity = 0.6981

Changing 12 month's r :
 price after change 1% of σ= 10.4949
 Vega = 9.3079
   >>> Weighted Sensitivity = 1.5358


#### Bucket內整合

In [29]:
ρ_option = np.exp( -0.01*np.abs(1-6/12)/np.min([1,6/12]) );ρ612 = np.min([ρ_delta*ρ_option,1]) 
FXR_WS_ν = np.sqrt( max( [ 0 , (FXR_WS_ν12)**2+(FXR_WS_ν6)**2 + 2*ρ612*FXR_WS_ν12*FXR_WS_ν6 ] ) )
print("   >>> EQR Vega  =",round(FXR_WS_ν,4))

   >>> EQR Vega  = 2.2286


##### 僅有一個bucket，故不需計算bucket間的彙整

### EQR Curvature

In [9]:
d60p_call,_ = Europrice((1+RW)*S,K,r9,σ9,T)
d60n_call,_ = Europrice((1-RW)*S,K,r9,σ9,T)

FXR_CVR_p = -(d60p_call-call-FXR_WS_Δ)
FXR_CVR_n = -(d60n_call-call+FXR_WS_Δ)
FXR_Curvature_Risk = max(max(FXR_CVR_p,0),max(FXR_CVR_n,0))

print("CRV+ =",round(FXR_CVR_p,4),"  CRV- =",round(FXR_CVR_n,4)) ; print()
print("K+ =",round(max(FXR_CVR_p,0),4),"  K- =",round(max(FXR_CVR_n,0),4)) ; print()
print("   >>> Curvature Risk =",round(FXR_Curvature_Risk,4))

CRV+ = -16.4778   CRV- = -24.9026

K+ = 0   K- = 0

   >>> Curvature Risk = 0


### FXR Capital Risk

In [10]:
FXR_mcr = FXR_WS_Δ + FXR_WS_ν + FXR_Curvature_Risk

print("Answer :") ; print()
print("    Minimum FXR Capital Requirements of this case is",round(FXR_mcr,4))

Answer :

    Minimum FXR Capital Requirements of this case is 37.5327


### GIRR Delta

In [32]:
##changing 6 month's r
d6r9 = (1/interpo(9,[6,1/(1+r6+0.0001)],[12,1/(1+r12)]))-1
dr_call6g,_ = Europrice(S,K,d6r9,σ9,T)
GIRR_Δ_6 = (dr_call6g-call)/0.0001
GIRR_WS6 = 0.017*GIRR_Δ_6/np.sqrt(2)
print("Changing 6 month's r :")
print(' price after change 1% of r=',round(dr_call6g,4))
print(" GIRR Delta =",round(GIRR_Δ_6,4)) 
print("   >>> Weighted Sensitivity =",round(GIRR_WS6,4))
print()

##changing 12 month's r
d12r9 = (1/interpo(9,[6,1/(1+r6)],[12,1/(1+r12+0.0001)]))-1
dr_call12g,_ = Europrice(S,K,d12r9,σ9,T)
GIRR_Δ_12 = (dr_call12g-call)/0.0001
GIRR_WS12 = 0.016*GIRR_Δ_12/np.sqrt(2)
print("Changing 12 month's r :")
print(' price after change 1% of r=',round(dr_call12g,4)) 
print(" GIRR Delta =",round(GIRR_Δ_12,4)) 
print("   >>> Weighted Sensitivity =",round(GIRR_WS12,4))

Changing 6 month's r :
 price after change 1% of r= 10.4036
 GIRR Delta = 17.9108
   >>> Weighted Sensitivity = 0.2153

Changing 12 month's r :
 price after change 1% of r= 10.4036
 GIRR Delta = 17.8064
   >>> Weighted Sensitivity = 0.2015


#### Bucket內整合

In [33]:
ρ_051 = 0.97
GIRR_WS_Δ = np.sqrt( max( [ 0 , (GIRR_WS12)**2+(GIRR_WS6)**2 + 2*ρ_051*GIRR_WS12*GIRR_WS6 ] ) )
print("   >>> GIRR Delta  =",round(GIRR_WS_Δ,4))

   >>> GIRR Delta  = 0.4136


### GIRR Curvature

In [35]:
d6r9p = (1/interpo(9,[6,1/(1+r6+0.017)],[12,1/(1+r12)]))-1 ;d6r9n = (1/interpo(9,[6,1/(1+r6-0.017)],[12,1/(1+r12)]))-1 
d017p_call6,_ = Europrice(S,K,d6r9p,σ9,T)
d017n_call6,_ = Europrice(S,K,d6r9n,σ9,T)
GIRR_CVR_p6 = -(d017p_call6 - call - GIRR_WS6)
GIRR_CVR_n6 = -(d017n_call6 - call + GIRR_WS6)
print("Changing 6 month's r :") ; print()
print("  CRV+ =",round(GIRR_CVR_p6,4)) 
print("  CRV- =",round(GIRR_CVR_n6,4)) 
print()
d12r9p = (1/interpo(9,[6,1/(1+r6)],[12,1/(1+r12+0.017)]))-1 ;d12r9n = (1/interpo(9,[6,1/(1+r6)],[12,1/(1+r12-0.017)]))-1
d017p_call12,_ = Europrice(S,K,d6r9p,σ9,T)
d017n_call12,_ = Europrice(S,K,d6r9n,σ9,T)
GIRR_CVR_p12 = -(d017p_call12 - call - GIRR_WS12)
GIRR_CVR_n12 = -(d017n_call12 - call + GIRR_WS12)
print("Changing 12 month's r :") ; print()
print("  CRV+ =",round(GIRR_CVR_p12,4))
print("  CRV- =",round(GIRR_CVR_n12,4)) 

Changing 6 month's r :

  CRV+ = -0.089
  CRV- = 0.0893

Changing 12 month's r :

  CRV+ = -0.1029
  CRV- = 0.1031


#### Bucket內整合

In [14]:
GIRR_k_p = np.sqrt( max([ 0 , max((GIRR_CVR_p6),0)**2 + max((GIRR_CVR_p12),0)**2 + (ρ_delta**2)*GIRR_CVR_p12*GIRR_CVR_p6*ϕ(GIRR_CVR_p12,GIRR_CVR_p6) ]) )
GIRR_k_n = np.sqrt( max([ 0 , max((GIRR_CVR_n6),0)**2 + max((GIRR_CVR_n12),0)**2 + (ρ_delta**2)*GIRR_CVR_n12*GIRR_CVR_n6*ϕ(GIRR_CVR_n12,GIRR_CVR_n6) ]) )
GIRR_k_c = max([ GIRR_k_p , GIRR_k_n ])
print("K+ = ",round(GIRR_k_p,4))
print("K- = ",round(GIRR_k_n,4))
print("   >>> Curvature 風險資本 =",round(GIRR_k_c,4))

K+ =  0.0
K- =  0.1667
   >>> Curvature 風險資本 = 0.1667


In [15]:
GIRR_mcr = GIRR_WS_Δ + GIRR_k_c

print("Answer :") ; print()
print("    Minimum GIRR Capital Requirements of this case is",round(GIRR_mcr,4))

Answer :

    Minimum GIRR Capital Requirements of this case is 0.5804


## Problem 2
如上設定，或再加入一大型"新興"經濟體電信股股票歐式陽春型"賣權"，賣出1000股部位。 \
 請問此部位需要計提多少的"FXR"市場風險資本? \
 又此部位需要計提多少的"GIRR"市場風險資本?

### EQR Delta

In [16]:
_,put = Europrice(S,K,r9,σ9,T)
_,dS_put = Europrice(1.01*S,K,r9,σ9,T)
FXR_Δp = (dS_put-put)/0.01
FXR_WS_Δp = RW*FXR_Δp

print("put price =",round(put,4)) ; print()
print('price after change 1% of stock price=',round(dS_call,4)) ; print()
print("Delta =",round(FXR_Δp,4)) ; print()
print("   >>> Weighted Sensitivity =",round(FXR_WS_Δp,4)) ; print()

put price = 8.4341

price after change 1% of stock price= 10.9902

Delta = -41.1599

   >>> Weighted Sensitivity = -24.6959



#### Bucket內整合

In [39]:
FXR_WS_Δ2 = np.sqrt( max( [ 0 , (FXR_WS_Δp)**2+(FXR_WS_Δ)**2 + 2*ρ_delta*FXR_WS_Δp*FXR_WS_Δ ] ) )
print("   >>> EQR Delta  =",round(FXR_WS_Δ2,4))

   >>> EQR Delta  = 10.69


### EQR Vega

In [37]:
d6_σ9 = interpo(9,[6,1.01*σ6],[12,σ12])
_,dσ_put6 = Europrice(S,K,r9,d6_σ9,T)
FXR_ν6p = (dσ_put6-put)/0.01
FXR_WS_ν6p = RW*FXR_ν6p*σ9
print("Changing 6 month's r :")
print(' price after change 1% of σ=',round(dσ_put6,4))
print(" Vega =",round(FXR_ν6p,4)) 
print("   >>> Weighted Sensitivity =",round(FXR_WS_ν6p,4))
print()
d12_σ9p = interpo(9,[6,1.01*σ6],[12,1.01*σ12])
_,dσ_put12 = Europrice(S,K,r9,d12_σ9,T)
FXR_ν12p = (dσ_put12-put)/0.01
FXR_WS_ν12p = RW*FXR_ν12p*σ9
print("Changing 12 month's r :") 
print(' price after change 1% of σ=',round(dσ_put12,4)) 
print(" Vega =",round(FXR_ν12p,4)) 
print("   >>> Weighted Sensitivity =",round(FXR_WS_ν12p,4))

Changing 6 month's r :
 price after change 1% of σ= 8.4764
 Vega = 4.2309
   >>> Weighted Sensitivity = 0.6981

Changing 12 month's r :
 price after change 1% of σ= 8.5271
 Vega = 9.3079
   >>> Weighted Sensitivity = 1.5358


#### Bucket內整合

In [38]:
ρ_option612 = np.exp( -0.01*np.abs(1-6/12)/np.min([1,6/12]) ); ρ_option66 = np.exp( 0 ); ρ_option1212 = np.exp( 0 )
ρ612 = np.min([ρ_delta*ρ_option612,1]) ; ρ66 = np.min([ρ_delta*ρ_option66,1]) ; ρ1212 = np.min([ρ_delta*ρ_option1212,1]) 

sigmaWS = (FXR_WS_ν12)**2+(FXR_WS_ν6)**2+(FXR_WS_ν12p)**2+(FXR_WS_ν6p)**2
sigmaWS = sigmaWS + 2*ρ612*(FXR_WS_ν12*FXR_WS_ν6 + FXR_WS_ν12*FXR_WS_ν6p + FXR_WS_ν12p*FXR_WS_ν6 + FXR_WS_ν12p*FXR_WS_ν6p)
sigmaWS = sigmaWS + 2*ρ66*(FXR_WS_ν6*FXR_WS_ν6p) + 2*ρ1212*(FXR_WS_ν12*FXR_WS_ν12p)

FXR_WS_ν2 = np.sqrt( max( [ 0 , sigmaWS] ) )
print("   >>> EQR Vega  =",round(FXR_WS_ν2,4))

   >>> EQR Vega  = 4.4567


### EQR Curvature

In [20]:
_,d60p_put = Europrice((1+RW)*S,K,r9,σ9,T)
_,d60n_put = Europrice((1-RW)*S,K,r9,σ9,T)

FXR_CVR_pp = -(d60p_put-put-FXR_WS_Δp)
FXR_CVR_np = -(d60n_put-put+FXR_WS_Δp)

CVR_plus = FXR_CVR_pp+FXR_CVR_p
CVR_minus = FXR_CVR_np+FXR_CVR_n

FXR_Curvature_Risk2 = max(max(CVR_plus,0),max(CVR_minus,0))

print("CRV+ =",round(CVR_plus,4),"  CRV- =",round(CVR_minus,4)) ; print()
print("K+ =",round(max(CVR_plus,0),4),"  K- =",round(max(CVR_minus,0),4)) ; print()
print("   >>> Curvature Risk =",round(FXR_Curvature_Risk2,4))

CRV+ = -32.9556   CRV- = -49.8051

K+ = 0   K- = 0

   >>> Curvature Risk = 0


In [21]:
FXR_mcr2 = FXR_WS_Δ2 + FXR_WS_ν2 + FXR_Curvature_Risk2

print("Answer :") ; print()
print("    Minimum FXR Capital Requirements of this case is",round(FXR_mcr2,4))

Answer :

    Minimum FXR Capital Requirements of this case is 15.1466


### GIRR Delta

In [40]:
_,dr_put6g = Europrice(S,K,d6r9,σ9,T)
GIRR_Δ_6_put = (dr_put6g - put)/0.0001
GIRR_WS6_put = 0.017*GIRR_Δ_6_put/np.sqrt(2)
print("Changing 6 month's r :")
print(' price after change 1% of r=',round(dr_put6g,4))
print(" GIRR Delta =",round(GIRR_Δ_6_put,4)) 
print("   >>> Weighted Sensitivity =",round(GIRR_WS6_put,4))
print()
_,dr_put12g = Europrice(S,K,d12r9,σ9,T)
GIRR_Δ_12_put = (dr_put12g - put)/0.0001
GIRR_WS12_put = 0.016*GIRR_Δ_12_put/np.sqrt(2)
print("Changing 12 month's r :")
print(" put price =",round(put,4)) 
print(' price after change 1% of r=',round(dr_put12g,4)) 
print(" GIRR Delta =",round(GIRR_Δ_12_put,4)) 
print("   >>> Weighted Sensitivity =",round(GIRR_WS12_put,4))

Changing 6 month's r :
 price after change 1% of r= 8.4322
 GIRR Delta = -18.9564
   >>> Weighted Sensitivity = -0.2279

Changing 12 month's r :
 put price = 8.4341
 price after change 1% of r= 8.4322
 GIRR Delta = -18.8459
   >>> Weighted Sensitivity = -0.2132


#### Bucket內整合

In [31]:
ρ_051 = 0.97 
ρ_11 = 1.00 
ρ_0505 = 1.00 
GIRR_WS_Δ2 = np.sqrt( max( [ 0 , (GIRR_WS12)**2+(GIRR_WS6)**2+(GIRR_WS6_put)**2+(GIRR_WS12_put)**2 
                               + 2*ρ_051*GIRR_WS12*GIRR_WS6 + 2*ρ_051*GIRR_WS12_put*GIRR_WS6 + 2*ρ_051*GIRR_WS12*GIRR_WS6_put + 2*ρ_051*GIRR_WS12_put*GIRR_WS6_put
                               + 2*ρ_11*GIRR_WS12*GIRR_WS12_put
                               + 2*ρ_0505*GIRR_WS6*GIRR_WS6_put] ) )
print("   >>> GIRR Delta  =",round(GIRR_WS_Δ2,4))

   >>> GIRR Delta  = 0.0241


### GIRR Curvature

In [41]:
_,d017p_put6 = Europrice(S,K,d6r9p,σ9,T)
_,d017n_put6 = Europrice(S,K,d6r9n,σ9,T)
GIRR_CVR_p6_put = -(d017p_put6 - put - GIRR_WS6_put)
GIRR_CVR_n6_put = -(d017n_put6 - put + GIRR_WS6_put)
print("for put, changing 6 month's r :") ; print()
print("  CRV+ =",round(GIRR_CVR_p6_put,4)) 
print("  CRV- =",round(GIRR_CVR_n6_put,4)) 
print()
_,d017p_put12 = Europrice(S,K,d6r9p,σ9,T)
_,d017n_put12 = Europrice(S,K,d6r9n,σ9,T)
GIRR_CVR_p12_put = -(d017p_put12 - put - GIRR_WS12_put)
GIRR_CVR_n12_put = -(d017n_put12 - put + GIRR_WS12_put)
print("for put ,changing 12 month's r :") ; print()
print("  CRV+ =",round(GIRR_CVR_p12_put,4))
print("  CRV- =",round(GIRR_CVR_n12_put,4)) 

for put, changing 6 month's r :

  CRV+ = 0.0875
  CRV- = -0.1016

for put ,changing 12 month's r :

  CRV+ = 0.1021
  CRV- = -0.1162


#### Bucket內整合

In [42]:
GIRR_CVR_p6_combine = GIRR_CVR_p6_put + GIRR_CVR_p6 ; GIRR_CVR_n6_combine = GIRR_CVR_n6_put + GIRR_CVR_n6 ; GIRR_CVR_p12_combine = GIRR_CVR_p12_put + GIRR_CVR_p12 ; GIRR_CVR_n12_combine = GIRR_CVR_n12_put + GIRR_CVR_n12
print("Changing 6 month's r :") ; print()
print("  CRV+ =",round(GIRR_CVR_p6_combine,4)) 
print("  CRV- =",round(GIRR_CVR_n6_combine,4)) 
print()
print("Changing 12 month's r :") ; print()
print("  CRV+ =",round(GIRR_CVR_p12_combine,4))
print("  CRV- =",round(GIRR_CVR_n12_combine,4)) 

Changing 6 month's r :

  CRV+ = -0.0015
  CRV- = -0.0123

Changing 12 month's r :

  CRV+ = -0.0007
  CRV- = -0.0131


In [43]:
GIRR_k_p2 = np.sqrt( max([ 0 , max((GIRR_CVR_p6_combine),0)**2 + max((GIRR_CVR_p12_combine),0)**2 + (ρ_delta**2)*GIRR_CVR_p12_combine*GIRR_CVR_p6_combine*ϕ(GIRR_CVR_p12_combine,GIRR_CVR_p6_combine) ]) )
GIRR_k_n2 = np.sqrt( max([ 0 , max((GIRR_CVR_n6_combine),0)**2 + max((GIRR_CVR_n12_combine),0)**2 + (ρ_delta**2)*GIRR_CVR_n12_combine*GIRR_CVR_n6_combine*ϕ(GIRR_CVR_n12_combine,GIRR_CVR_n6_combine) ]) )
GIRR_k_c2 = max([ GIRR_k_p2 , GIRR_k_n2 ])
print("K+ = ",round(GIRR_k_p2,4))
print("K- = ",round(GIRR_k_n2,4))
print("   >>> Curvature 風險資本 =",round(GIRR_k_c2,4))

K+ =  0.0
K- =  0.0
   >>> Curvature 風險資本 = 0.0


In [27]:
GIRR_mcr2 = GIRR_WS_Δ2 + GIRR_k_c2

print("Answer :") ; print()
print("    Minimum GIRR Capital Requirements of this case is",round(GIRR_mcr2,4))

Answer :

    Minimum GIRR Capital Requirements of this case is 0.0241
